In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd

/home/den/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('big_table.csv', sep=',')

In [3]:
data[25:29]

,Unnamed: 0,drug_cod,drug_x,provider_code,provider_drug_cod,code,drug_y
25,25,9953,Ацетилсалициловая к-та 500мг №10 тб,0000000000000000000013561,5163,13561,"Ацетилсалициловая к-та таб 0,5 г №10"
26,26,10604,Сульфокамфокаин 100мг/мл 2мл №10 амп р-р д/ин,0000000000000000000010250,5163,10250,Сульфокамфокаин амп 2 мл №10
27,27,11805,Пирацетам 1200мг №20 тб пл/о,0000000000000000000017469,5163,17469,Пирацетам таб 1200 мг № 20 п/о
28,28,3941,Торвакард 20мг №30 тб пл/о,0000000000000000000017256,5163,17256,"Торвакард 0,02 №30 таб. п\о"


In [4]:
Parma_Sentence = data.drug_x.tolist()
Parma_drug_cod = data.drug_cod.tolist()
Other_Sentence = data.drug_y.tolist()
#Parma_Sentence = ['?' + DrugName.strip()  + '^' for DrugName in Parma_Sentence]
#Other_Sentence = ['?' + DrugName.strip()  + '^' for DrugName in Other_Sentence]
Parma_Sentence = [DrugName.strip()  for DrugName in Parma_Sentence]
Other_Sentence = [DrugName.strip()  for DrugName in Other_Sentence]
Parma_Sentence = ['\t' +DrugName.split('\t')[0] + '\n' for DrugName in Parma_Sentence]
Other_Sentence = ['\t' +DrugName.split('\t')[0] + '\n' for DrugName in Other_Sentence]
print('Количество строк в списке Парма Медикал:', len(Parma_Sentence), sep = ' ')
print('Количество строк в списке сторонних организаций:', len(Other_Sentence), sep = ' ')
PM_unik_drug = set()
for drug_line in Parma_drug_cod:
    if (drug_line not in PM_unik_drug):
        PM_unik_drug.add(drug_line)
PM_unik_drug = sorted(list(PM_unik_drug))    
PM_unik_drug = set(PM_unik_drug)
print('Количество уникальных препаратов Парма Медикал:',len(PM_unik_drug), sep = ' ')
nb_samples = len(Other_Sentence)

Количество строк в списке Парма Медикал: 20206
Количество строк в списке сторонних организаций: 20206
Количество уникальных препаратов Парма Медикал: 7230


In [5]:
PM_chars = set()
for drug_line in Parma_Sentence:
    for ch in drug_line:
        if (ch not in PM_chars):
            PM_chars.add(ch)
PM_chars = sorted(list(PM_chars))    
PM_chars = set(PM_chars)
display('Словарь Парма Медикал: ' + str(len(PM_chars)) + " символа")

'Словарь Парма Медикал: 143 символа'

In [6]:
Other_chars = set()
for drug_line in Other_Sentence:
    for ch in drug_line:
        if (ch not in Other_chars):
            Other_chars.add(ch)
Other_chars = sorted(list(Other_chars))
Other_chars = set(Other_chars)
display('Внешний словарь: ' + str(len(Other_chars)) + " символа")

'Внешний словарь: 153 символа'

In [7]:
PM_index_to_char_dict = {}
PM_char_to_index_dict = {}

for k, v in enumerate(PM_chars):
    PM_index_to_char_dict[k] = v
    PM_char_to_index_dict[v] = k

In [8]:
Other_index_to_char_dict = {}
Other_char_to_index_dict = {}

for k, v in enumerate(Other_chars):
    Other_index_to_char_dict[k] = v
    Other_char_to_index_dict[v] = k

In [9]:
max_len_PM_sent = max([len(line) for line in Parma_Sentence])
max_len_Other_sent = max([len(line) for line in Other_Sentence])

In [21]:
tokenized_Other_sentences = np.zeros(shape = (nb_samples,max_len_Other_sent,len(Other_chars)), dtype='float32')

"""encoder_input = Input(shape=(None,len(Other_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]"""

"""decoder_input = Input(shape=(None,len(PM_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(PM_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)"""

json_file = open('my_model_no__BosEos_1.json', 'r')
lmodel = json_file.read()
json_file.close()

from keras.models import model_from_json
my_model = model_from_json(lmodel)

my_model.load_weights('my_model_no__BosEos_1.h5')

my_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [16]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(PM_chars)))
    target_seq[0, 0,PM_char_to_index_dict[' ']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_PM_char = PM_index_to_char_dict[max_val_index]
        translated_sent += sampled_PM_char
        
        if ( (sampled_PM_char == '\n') or (len(translated_sent) > max_len_PM_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(PM_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent

In [22]:
encoder_input = Input(shape=(None,len(Other_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]
encoder_model_inf = Model(encoder_input, encoder_states)

decoder_input = Input(shape=(None,len(PM_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(PM_chars),activation='softmax')
decoder_out = decoder_dense (decoder_out)

json_file = open('my_model_no__BosEos_1.json', 'r')
lmodel = json_file.read()
json_file.close()

from keras.models import model_from_json
my_model = model_from_json(lmodel)
my_model.load_weights('my_model_no__BosEos_1.h5')
my_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

for seq_index in range(30):
    inp_seq = tokenized_Other_sentences[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-')
    print('Input sentence:', Other_Sentence[seq_index])
    print('Decoded sentence:', translated_sent)

-
Input sentence: 	Имудон таб д/рассас №24

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШ
-
Input sentence: 	Глаумакс 0,005% 2,5 мл фл-кап гл капли

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШ
-
Input sentence: 	Эссенциале-форте Н капс 300 мг №30.

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШ
-
Input sentence: 	Эссенциале-форте Н капс 300 мг №30.

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШ
-
Input sentence: 	Димедрол таб №10

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШ
-
Input sentence: 	Ибупрофен гель 5% 50г туба

Decoded sentence: И****иII'IIьIШьIIШьIШсIьIШьIШ888ll>l>Cллxл&ллЛЛЗЛЗcчыиIIьIШьIШсIьIШьIШ888